# Seleção de Features usando o teste chi quadrado de independência

## 1. Carregando módulos e dados

In [2]:
# Manipulação dos dados
import numpy  as np
import scipy  as sp
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn           as sns
%matplotlib inline

In [3]:
# Carregando os dados
data = pd.read_excel('../../Dados/Banco labels.xlsx')

In [4]:
# Visualização da tabela
data.head(5)

,TB_desfecho_final_Dri,Tbafranio,Obs_desfechoTB_Dri_Dani,Data_inclusão,Nome,Desfecho_tuberculose_lista_Mauro,Desfecho_TB_162_Revisão_ADRIANA,Observaça0_desfecho_TB,Resultado_BAAR_1,Resultado_BAAR_2,...,Perdeu_aula_hoje_CA,Deixou_de_ganhar_dinheiro_hoje_CA,Quanto_deixou_ganhar_CA,Caso_acompanhado_quem_custeou_tudo_CA,Valor_CA,Avaliador_custo_CA,Data_avaliação_custo_CA,Resultado_cultura_final < 3 & Realizou_RX = 1 (FILTER),ANO de inclusao,Novocriteriotbafranio
0,TB,SIM,NaN,2008-04-03,NaN,9,TB,TB PROBABILIDADE,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2008,1
1,TB,SIM,NaN,2006-10-03,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2006,1
2,TB,SIM,NaN,2007-06-21,NaN,TB,TB,TB CONFIRMADA,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2007,1
3,TB,SIM,NaN,2009-09-30,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2009,1
4,TB,SIM,NaN,2009-06-19,NaN,TB,NaN,NaN,Negativo,Negativo,...,Não,Não,"""Não se aplica""",não se aplica,não se aplica,Marcia dos Santos,2009-06-19,Not Selected,2009,1


In [5]:
# Alterando o nome da variável TB_desfecho_final_Dri para desfecho
data.rename(columns={'TB_desfecho_final_Dri':'desfecho'},inplace=True)

In [6]:
# Carregando as features salvas na seleção inicial de variáveis
import pickle
features_selecionadas = pickle.load(open('../../Dados/features_filtradas.pkl', 'rb'))

In [11]:
# Exibindo as variáveis salvas
print(features_selecionadas)

# Criando um dataframe a partir das features pré-selecionadas
dados = data[features_selecionadas].copy()

['desfecho', 'TTO_anterior_TB_triagem_enfermeiro', 'Cicatriz_BCG', 'Idade', 'Sexo', 'Raça', 'Estado_civil', 'Tem_companheiro', 'Peso_habitual', 'Peso_atual', 'Altura_cm', 'Tosse', 'Semanas_tosse', 'Expectoração', 'Semanas_expectoração', 'Hemoptóicos', 'Semanas_hemoptóicos', 'Hemoptise', 'Semanas_hemoptise', 'Quantidade_hemoptise', 'Sudorese_noturna', 'Semanas_sudorese', 'Febre', 'Semanas_febre', 'Dispnéia', 'Dispnéia_semanas', 'Perda_de_apetite', 'Perda_apetite_semanas', 'Perda_peso_10percent', 'Perda_peso_semanas', 'Dor_torácica', 'Dor_torácica_semanas', 'Internação_hospital_2anos', 'Prisão_2anos', 'Delegacia_2anos', 'Contato_TBP_2anos', 'Fuma', 'Número_cigarros_dia', 'CAGE']


### 1.1 Pré-processamento dos dados

In [12]:
# Alterando os labels dos desfechos
dados.desfecho.replace(['TB', 'Não TB','IGN',8],['TB+','TB-','desconhecido','desconhecido'],inplace=True)

# Selecionando os desfechos que possuem diagnóstico conhecido: TB+ e TB-
dados = dados[(dados.desfecho == 'TB+') | (dados.desfecho == 'TB-')]

In [13]:
# Quantidade_hemoptise: Poucos valores significantes
dados.drop(['Quantidade_hemoptise'],axis=1,inplace=True)

# Peso atual/habitual: Não importa para o diagnóstico
dados.drop(['Peso_atual'],axis=1,inplace=True)
dados.drop(['Peso_habitual'],axis=1,inplace=True)

# Altura: Não importa para o diagnóstico
dados.drop(['Altura_cm'],axis=1,inplace=True)

# Número de cigarros
dados.drop(['Número_cigarros_dia'],axis=1,inplace=True)

# Número de cigarros
dados.drop(['Perda_apetite_semanas'],axis=1,inplace=True)

In [14]:
# Renomeando os valores IGN por ignorado
dados.replace(['IGN'],'ignorado',inplace=True)

# Substituindo os valores da variável Sexo
dados.Sexo.replace([8,2],['ignorado','ignorado'],inplace=True)
                    
# Substituindo os valores da variável Raça
dados.Raça.replace([8],'ignorado',inplace=True)
                    
# Substituindo os valores da variável Estado_civil
dados.Estado_civil.replace([np.nan, 8, 0],['ignorado','ignorado','ignorado'],inplace=True)        

# Substituindo os valores da variável Raça
dados.Raça.replace([8],'ignorado',inplace=True)

# Substituindo os valores da variável Semanas_tosse
dados.Semanas_tosse.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável Semanas_expectoração
dados.Semanas_expectoração.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável Semanas_tosse
dados.Semanas_hemoptóicos.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável Semanas_hemoptise
dados.Semanas_hemoptise.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável 
dados.Sudorese_noturna.replace([9],'ignorado',inplace=True)

# Substituindo os valores da variável 
dados.Semanas_sudorese.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável 
dados.Semanas_febre.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável 
dados.Dispnéia.replace([2],['ignorado'],inplace=True)

# Substituindo os valores da variável 
dados.Dispnéia_semanas.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável 
dados.Perda_peso_semanas.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

# Substituindo os valores da variável 
dados.Dor_torácica_semanas.replace(['não se aplica','crônico+sempre'],[0,180],inplace=True)

In [15]:
# Features presentes
print(dados.columns)

Index(['desfecho', 'TTO_anterior_TB_triagem_enfermeiro', 'Cicatriz_BCG',
       'Idade', 'Sexo', 'Raça', 'Estado_civil', 'Tem_companheiro', 'Tosse',
       'Semanas_tosse', 'Expectoração', 'Semanas_expectoração', 'Hemoptóicos',
       'Semanas_hemoptóicos', 'Hemoptise', 'Semanas_hemoptise',
       'Sudorese_noturna', 'Semanas_sudorese', 'Febre', 'Semanas_febre',
       'Dispnéia', 'Dispnéia_semanas', 'Perda_de_apetite',
       'Perda_peso_10percent', 'Perda_peso_semanas', 'Dor_torácica',
       'Dor_torácica_semanas', 'Internação_hospital_2anos', 'Prisão_2anos',
       'Delegacia_2anos', 'Contato_TBP_2anos', 'Fuma', 'CAGE'],
      dtype='object')


In [20]:
# Substituindo os valores por nulos 
dados.replace(['ignorado'],np.nan,inplace=True)

# Remover todos os casos ignorados
dados = dados.dropna(how='any')

## 2. Seleção de variáveis utilizando o teste do chi quadrado de independência

O teste do chi quadrado verifica se existe a relação entre duas variáveis categóricas. Dessa forma, será verificada a relação entre as variáveis presentes e o desfecho, ou seja, se o atributo possui alguma relação com o diagnóstico da doença ou não. 

O teste do Chi-quadrado é realizado para a verificação de independência entre duas variáveis categóricas.

- Hipótese nula $H_0$ : As variáveis são independentes.
- Hipótese alternativa $H_a$: As variáveis são dependentes

In [24]:
# 
from scipy.stats import chi2_contingency

# Criando um DataFrame para o armazenamento dos valores

dict_features = dict()

for feature in list(dados.columns)[1:]:
    
    if dados[feature].dtype != 'O':
        continue
    
    print('Tabela de Contingência entre o desfecho e {}\n'.format(feature))
    
    # Contingency table from pandas
    cross_tab = pd.crosstab(dados[feature],dados['desfecho'],margins=True)
    print(pd.crosstab(dados[feature],dados['desfecho'],margins=True))
    print()
    
    # Chi Squared Test 
    chi2, p, dof, expected =  chi2_contingency(cross_tab)
    print('Chi2: {}\tp-value: {}\tDOF: {}\nExpected_val: \n{}\n\n'.format(chi2,p,dof,expected))
    
    dict_features[feature] = [chi2,p,dof,(len(dados[feature].unique()) - 1),]

Tabela de Contingência entre o desfecho e TTO_anterior_TB_triagem_enfermeiro

desfecho                            TB+  TB-   All
TTO_anterior_TB_triagem_enfermeiro                
Não                                 329  698  1027
Sim                                  77  222   299
All                                 406  920  1326

Chi2: 4.302714575568573	p-value: 0.36658529994135014	DOF: 4
Expected_val: 
[[  314.45098039   712.54901961  1027.        ]
 [   91.54901961   207.45098039   299.        ]
 [  406.           920.          1326.        ]]


Tabela de Contingência entre o desfecho e Cicatriz_BCG

desfecho      TB+  TB-   All
Cicatriz_BCG                
Não           135  422   557
Sim           271  498   769
All           406  920  1326

Chi2: 18.41110548169797	p-value: 0.0010254530312541828	DOF: 4
Expected_val: 
[[  170.54449472   386.45550528   557.        ]
 [  235.45550528   533.54449472   769.        ]
 [  406.           920.          1326.        ]]


Tabela de Contingê

In [25]:
# Criando um DataFrame a partir do dicionário criado anteriormente
chi2_dict = pd.DataFrame.from_dict(dict_features,orient='index')
chi2_dict.columns = ['chi2','pval','dof','dof_real']

# Dataframe criado
chi2_dict

,chi2,pval,dof,dof_real
CAGE,0.337220,9.872865e-01,4,1
Hemoptóicos,1.580291,8.123301e-01,4,1
Febre,36.326449,2.479022e-07,4,1
Cicatriz_BCG,18.411105,1.025453e-03,4,1
Perda_peso_10percent,104.706610,9.781601e-22,4,1
Tem_companheiro,2.831757,5.863634e-01,4,1
Dispnéia,7.485618,1.123452e-01,4,1
Internação_hospital_2anos,0.474640,9.759223e-01,4,1
Prisão_2anos,21.013432,3.147333e-04,4,1
Estado_civil,34.622730,3.131765e-05,8,3


In [36]:
variaveis_dependentes = chi2_dict[chi2_dict.pval < 0.05]
variaveis_independentes = chi2_dict[chi2_dict.pval > 0.05]

In [37]:
variaveis_dependentes

,chi2,pval,dof,dof_real
Febre,36.326449,2.479022e-07,4,1
Cicatriz_BCG,18.411105,1.025453e-03,4,1
Perda_peso_10percent,104.706610,9.781601e-22,4,1
Prisão_2anos,21.013432,3.147333e-04,4,1
Estado_civil,34.622730,3.131765e-05,8,3
Sexo,26.531106,2.472477e-05,4,1
Delegacia_2anos,19.281812,6.918093e-04,4,1
Sudorese_noturna,16.728621,2.182237e-03,4,1


In [38]:
variaveis_independentes

,chi2,pval,dof,dof_real
CAGE,0.337220,0.987287,4,1
Hemoptóicos,1.580291,0.812330,4,1
Tem_companheiro,2.831757,0.586363,4,1
Dispnéia,7.485618,0.112345,4,1
Internação_hospital_2anos,0.474640,0.975922,4,1
Dor_torácica,0.187133,0.995886,4,1
Raça,5.683932,0.224029,4,1
Hemoptise,1.465786,0.832684,4,1
TTO_anterior_TB_triagem_enfermeiro,4.302715,0.366585,4,1
Fuma,1.809072,0.936394,6,2
